In [ ]:
import bgheatmaps as bgh
from bgheatmaps.slicer import Slicer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys, os
import pandas as pd
sys.path.append(r'C:\Users\Valentino\Documents\Python\wholeBrain_PNN_analysis')
# Custom packages
from wholebrain_tools import dataIO, aba

paths = dataIO.pathParser()
structuresFile = paths.structures
A = aba.Atlas(nodes=structuresFile)


# Load Data

In [ ]:
midDf = pd.read_csv('stain.csv', header=[0], index_col=[0])
# midDf = midDf.drop(1009)       # Drop Fiber tracts
midDf['acronym'] = A.ids_to_acronyms(midDf.index.get_level_values('mid'))


#### Set A-P levels for the heatmaps and the vmax

In [ ]:
ap = [3227.27272727,6681.81818182, 10356]
VMAX = 2

# Staining - Render the brain

In [ ]:
metric = 'diffFluo'

bgData = midDf[['acronym',metric]].set_index('acronym')
dataDict = bgData[metric].to_dict()

print("Rendering...")
heatmap = bgh.heatmap(
    dataDict,
    position=5000,
    orientation="frontal",
    thickness=15000,
    title="WFA diffuse",
    cmap='PuBu',
    vmin=0,
    vmax=VMAX,
    format="2D",
)
heatmap.scene.close()

# Staining - Slice and save all images

In [ ]:
OUTPUT_FOLDER = os.path.join(os.getcwd(), 'render_background')

if not os.path.isdir(OUTPUT_FOLDER):
    os.mkdir(OUTPUT_FOLDER)
else:
    raise Exception('OUTPUT_FOLDER already present!')

for coord in ap:
    s = Slicer(coord, "frontal", 100, heatmap.scene.root)
    heatmap.slicer = s

    projected, _ = heatmap.slicer.get_structures_slice_coords(
                heatmap.regions_meshes, heatmap.scene.root
            )

    f, ax = plt.subplots(figsize=(9, 9))
    for r, coords in projected.items():
        name, segment = r.split("_segment_")
        ax.fill(
            coords[:, 0],
            coords[:, 1],
            color=heatmap.colors[name],
            label=name if segment == "0" and name != "root" else None,
            lw=1,
            ec="k",
            zorder=-1 if name == "root" else None,
            alpha=0.3 if name == "root" else None,
        )

    # style axes
    ax.invert_yaxis()
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set(xlabel='', ylabel='')
    ax.set_aspect('equal',adjustable='box')

    # Only the left part
    ax.set_xlim([0,5700])

    fName = f"wfaStain_{coord:05.0f}.svg"
    f.savefig(os.path.join(OUTPUT_FOLDER, fName))

# Background - Render the brain

In [ ]:
midDf = pd.read_csv('negative_control.csv', header=[0], index_col=[0])
# midDf = midDf.drop(1009)       # Drop Fiber tracts
midDf['acronym'] = A.ids_to_acronyms(midDf.index.get_level_values('mid'))


In [ ]:
metric = 'diffFluo'

bgData = midDf[['acronym',metric]].set_index('acronym')
dataDict = bgData[metric].to_dict()

print("Rendering...")
heatmap = bgh.heatmap(
    dataDict,
    position=5000,
    orientation="frontal",
    thickness=15000,
    title="WFA diffuse",
    cmap='PuBu',
    vmin=0,
    vmax=VMAX,
    format="2D",
)
heatmap.scene.close()

# Background - Slice and save all images

In [ ]:
for coord in ap:
    s = Slicer(coord, "frontal", 100, heatmap.scene.root)
    heatmap.slicer = s

    projected, _ = heatmap.slicer.get_structures_slice_coords(
                heatmap.regions_meshes, heatmap.scene.root
            )

    f, ax = plt.subplots(figsize=(9, 9))
    for r, coords in projected.items():
        name, segment = r.split("_segment_")
        ax.fill(
            coords[:, 0],
            coords[:, 1],
            color=heatmap.colors[name],
            label=name if segment == "0" and name != "root" else None,
            lw=1,
            ec="k",
            zorder=-1 if name == "root" else None,
            alpha=0.3 if name == "root" else None,
        )

    # style axes
    ax.invert_yaxis()
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)

    ax.spines["left"].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set(xlabel='', ylabel='')
    ax.set_aspect('equal',adjustable='box')

    # Only the right part
    ax.set_xlim([5700,11400])


    fName = f"wfaNC_{coord:05.0f}.svg"
    f.savefig(os.path.join(OUTPUT_FOLDER, fName))